# Simon's Travel Tracker
### Testing out Folium for Geovisualization

In [159]:
import folium
import pandas
from geopy.geocoders import Nominatim
import branca.colormap as cm

from plugins import FullScreen
locations_dir = './locations/'
cities_list = 'cities.csv'

In [160]:
# Get Lat and Long of cities
lats_longs = []
cities = []
df = pandas.read_csv('{}{}'.format(locations_dir, cities_list), sep=',')
rows = df.shape[0]

for x in range(rows):
    cities.append(df.iloc[x]['City'])

geolocator = Nominatim()
for city in cities:
    location = geolocator.geocode(city)
    lats_longs.append([location.latitude, location.longitude])
    

In [161]:
# Sanity check
for result in lats_longs:
    print(result[0], result[1])

29.7589382 -95.3676974
29.4246002 -98.4951405
33.5778631 -101.8551665
32.7762719 -96.7968559
29.299328 -94.7945882
36.1662859 -115.149225
34.054935 -118.244476
22.2793278 114.1628131
39.9059631 116.391248
31.2253441 121.4888922
34.6968642 139.4049033
9.0 38.75
11.122604 39.634982
52.3745403 4.89797550561798


In [164]:
# Note: Need to confirm differente between add_child and add_to

f = folium.Figure(width=1000, height=500) # Instantiate a figure
travel_map = folium.Map(location=[0, 0], tiles='openstreetmap', zoom_start=2, min_zoom=2).add_to(f) # Instantiate a world map and add it to the figure
FullScreen.Fullscreen(title='Travel Tracker', force_separate_button=True).add_to(travel_map) # Add a Fullscreen button to the map

travel_map.add_child(folium.features.LatLngPopup()) # Add Latitude Longitude Popup to the map
#travel_map.add_child(folium.features.PolyLine([31,-99]))

# Colorscale: Steps of 10 between 0 and 200
colorscale = cm.linear.Set1.scale(0, 100).to_step(5)
colorscale.caption = 'Total number of days spent in each location'
travel_map.add_child(colorscale)

# Define locations: (Location, Years visited, total days visited, cities)
my_locations = [('CA', [2013], 7, "Los Angeles"),
                ('CHN', [2017], 14, "Beijing & Shanghai"),
                ('ETH', [2008, 2011], 105, "Addis Ababa & Dessie"),
                ('HK', [2017], 112, "Hong Kong"),
                ('JPN', [2017], 7, "Tokyo"),
                ('NLD', [2008, 2011], 2, "Holland"),
                ('NV', [2013], 7, "Las Vegas"),
                ('TX', [1996], -1, "I live here and I hate it")]

# Add GeoJson shapes to the map
map_countries = folium.FeatureGroup(name='Countries') # Create a feature group for the countries
for location in my_locations:
    map_loc = folium.GeoJson(open('{}{}.geo.json'.format(locations_dir, location[0]), encoding = "utf-8-sig").read(), 
                             style_function=lambda x: {'fillColor': 'yellow', 'dashArray': '5, 5'}, 
                             highlight_function=lambda x: {'weight': 2, 'fillColor': 'blue'}) # Load the .geo.json data and customize
    
    map_loc.add_child(folium.Popup(location[3])) # Shows the name of the cities that have been visited within this location on click
    map_countries.add_child(map_loc) # Add the GeoJSON to the map_countries feature group
    
map_countries.add_to(travel_map) # Add the feature group to the map

# Add Markers
map_cities = folium.FeatureGroup(name='Cities') # Create a feature group for the cities
for x in range(rows):
    mkr = folium.Marker([lats_longs[x][0], 
                         lats_longs[x][1]], 
                        popup=df.iloc[x]['City'], 
                        icon=folium.Icon(color='red', icon='info-sign')) # Add a marker to show the name of the city located at the marker on click
    
    map_cities.add_child(mkr) # Add the marker to the feature group

map_cities.add_to(travel_map) # Add the feature group to the map


NameError: name 'zoom_start_defined' is not defined

In [163]:
# Export and preview
folium.LayerControl().add_to(travel_map) # Add Layer controls - Must add this after feature groups have been created?
travel_map